# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [48]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import geoviews as gv
import geoviews.feature as gf
from geoviews import dim

# Import API key
from api_keys import geoapify_key

In [49]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,bubaque,11.2833,-15.8333,24.43,54,100,6.87,GW,1675655109
1,1,arcelia,18.2833,-100.2667,24.14,37,0,1.46,MX,1675655452
2,2,wenchi,7.7392,-2.1046,23.51,77,84,1.81,GH,1675655452
3,3,samarai,-10.6167,150.6667,27.40,83,99,5.80,PG,1675655453
4,4,iranshahr,27.2025,60.6848,12.19,71,0,2.06,IR,1675655453


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [63]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE
city_data_df.hvplot.points('Lng', 'Lat', geo=True,size="Humidity", color='City', alpha=0.5,
                        tiles='EsriTerrain',height=400, width=899)
# Display the map
# YOUR CODE HERE

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [56]:
# Narrow down cities that fit criteria and drop any results with null values
# Drop any rows with null values
city_data_df2=city_data_df.dropna()

# A max temperature lower than 27 degrees but higher than 21
city_data_df3=city_data_df2[(city_data_df2["Max Temp"]>21) | (city_data_df2["Max Temp"]<27) ]

# Wind speed less than 4.5 m/s
city_data_df4=city_data_df3[city_data_df3["Wind Speed"]<4.5]

# Zero cloudiness
final_data=city_data_df4[city_data_df4["Cloudiness"]==0]

# Display sample data
final_data

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
1,1,arcelia,18.2833,-100.2667,24.14,37,0,1.46,MX,1675655452
4,4,iranshahr,27.2025,60.6848,12.19,71,0,2.06,IR,1675655453
8,8,chiredzi,-21.0500,31.6667,19.29,75,0,0.90,ZW,1675655455
9,9,bethel,41.3712,-73.4140,4.25,62,0,0.00,US,1675655456
10,10,lazaro cardenas,17.9583,-102.2000,23.10,82,0,1.43,MX,1675655157
...,...,...,...,...,...,...,...,...,...,...
526,526,banmankhi,25.8833,87.1833,20.74,46,0,3.55,IN,1675655830
530,530,coihueco,-36.6167,-71.8333,18.20,53,0,2.61,CL,1675655833
531,531,roquetas de mar,36.7642,-2.6148,9.23,70,0,4.24,ES,1675655834
539,539,san ignacio,17.1561,-89.0714,21.08,94,0,0.00,BZ,1675655840


### Step 3: Create a new DataFrame called `hotel_df`.

In [60]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df=final_data.loc[:, ("City","Country","Lat","Lng","Humidity")]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"]=""

# Display sample data
# YOUR CODE HERE
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
1,arcelia,MX,18.2833,-100.2667,37,
4,iranshahr,IR,27.2025,60.6848,71,
8,chiredzi,ZW,-21.0500,31.6667,75,
9,bethel,US,41.3712,-73.4140,62,
10,lazaro cardenas,MX,17.9583,-102.2000,82,
...,...,...,...,...,...,...
526,banmankhi,IN,25.8833,87.1833,46,
530,coihueco,CL,-36.6167,-71.8333,53,
531,roquetas de mar,ES,36.7642,-2.6148,70,
539,san ignacio,BZ,17.1561,-89.0714,94,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [61]:
# Set parameters to search for a hotel
radius = 10000


# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    # YOUR CODE HERE
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    #params["filter"] = # YOUR CODE HERE
    #params["bias"] = # YOUR CODE HERE
    parameters = {
    "categories":"accommodation.hotel",
    "lat": hotel_df["Lat"][index],
    "lon": hotel_df["Lng"][index],
    "range": "<=10000",
    "apiKey": geoapify_key
    }
    
    # Set base URL
    #base_url = "https://api.geoapify.com/v2/places"
    base_url ="https://api.geoapify.com/v2/places?"

    # Make and API request using the params dictionaty
    #name_address = # YOUR CODE HERE
    
    response = requests.get(url=base_url, params=parameters)
    
    # Convert the API response to JSON format
    name_address = response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
arcelia - nearest hotel: Finca del Alba
iranshahr - nearest hotel: هتل قصِر
chiredzi - nearest hotel: Nesbitt Arms Hotel
bethel - nearest hotel: Hampton Inn Danbury
lazaro cardenas - nearest hotel: Hotel Sol del Pacífico
ushuaia - nearest hotel: Apart Hotel Aires del Beagle
ukiah - nearest hotel: Palace Hotel
yulara - nearest hotel: Outback Pioneer Hotel & Lodge
mar del plata - nearest hotel: Nuevo Ostende
brewster - nearest hotel: Comfort Inn
san patricio - nearest hotel: Best Western Plus
salalah - nearest hotel: Muscat International Hotel
laguna - nearest hotel: Holiday Inn Express & Suites
evanston - nearest hotel: Margarita European Inn
khash - nearest hotel: No hotel found
vila velha - nearest hotel: Hotel Prainha
comodoro rivadavia - nearest hotel: No hotel found
lata - nearest hotel: Residencial Botânico
caravelas - nearest hotel: Pousada dos Navegantes
copala - nearest hotel: Hotel El Buzo
chuy - nearest hotel: Alerces
meriden - nearest hotel: Comfort Inn

,City,Country,Lat,Lng,Humidity,Hotel Name
1,arcelia,MX,18.2833,-100.2667,37,Finca del Alba
4,iranshahr,IR,27.2025,60.6848,71,هتل قصِر
8,chiredzi,ZW,-21.0500,31.6667,75,Nesbitt Arms Hotel
9,bethel,US,41.3712,-73.4140,62,Hampton Inn Danbury
10,lazaro cardenas,MX,17.9583,-102.2000,82,Hotel Sol del Pacífico
...,...,...,...,...,...,...
526,banmankhi,IN,25.8833,87.1833,46,Hotel Holiday International
530,coihueco,CL,-36.6167,-71.8333,53,Hotel Argentina
531,roquetas de mar,ES,36.7642,-2.6148,70,Pensión Juan Pedro
539,san ignacio,BZ,17.1561,-89.0714,94,Rainforest Haven Inn


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [64]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE
hotel_df.hvplot.points('Lng', 'Lat', geo=True,size="Humidity", color='City', alpha=0.6,
                        tiles='EsriTerrain',height=400, width=899, hover_cols=["City", "Country", "Hotel Name"])
# Display the map
# YOUR CODE HERE

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)